# AAPL options prediction

In [1]:
#Import required libraries
import torch
import torch.nn as nn
import torch.nn.init as init
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import CyclicLR
import math
import numpy as np
import pandas as pd
from collections import OrderedDict
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import tqdm as notebook_tqdm
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import griddata
import torch.nn.functional as F

In [2]:
# Choose the device for computation
device = torch.device('cpu')

## Data Preprocessing

In [3]:
# Methods to calculate the Black-Scholes option prices

def black_scholes_call(S, K, T, r, sigma):
    """
    Calculate the Black-Scholes call option price.

    Parameters:
    S (float): Current stock price
    K (float): Strike price
    T (float): Time to expiration (in years)
    r (float): Risk-free interest rate
    sigma (float): Volatility of the underlying stock

    Returns:
    float: Call option price
    """
    d1 = (torch.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * torch.sqrt(T))
    d2 = d1 - sigma * torch.sqrt(T)
    
    N_d1 = 0.5 * (1 + torch.erf(d1 / math.sqrt(2)))
    N_d2 = 0.5 * (1 + torch.erf(d2 / math.sqrt(2)))
    
    call_price = S * N_d1 - K * torch.exp(-r * T) * N_d2
    return call_price

def black_scholes_put(S, K, T, r, sigma):
    """
    Calculate the Black-Scholes put option price.

    Parameters:
    S (float): Current stock price
    K (float): Strike price
    T (float): Time to expiration (in years)
    r (float): Risk-free interest rate
    sigma (float): Volatility of the underlying stock

    Returns:
    float: Put option price
    """
    d1 = (torch.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * torch.sqrt(T))
    d2 = d1 - sigma * torch.sqrt(T)
    
    N_d1 = 0.5 * (1 + torch.erf(-d1 / math.sqrt(2)))
    N_d2 = 0.5 * (1 + torch.erf(-d2 / math.sqrt(2)))
    
    put_price = K * torch.exp(-r * T) * N_d2 - S * N_d1
    return put_price


In [4]:
options_data = pd.read_csv('jnnksdxiidjlejyn.csv')
stock_prices_data = pd.read_csv('stock_prices_aapl_2022.csv')

In [5]:
options_data.shape

(231745, 19)

In [6]:
options_data.describe()

,strike_price,best_bid,best_offer,volume,impl_volatility,delta,gamma,vega,theta,optionid,contract_size,index_flag
count,231745.000000,231745.000000,231745.000000,231745.000000,211076.000000,211076.000000,211076.000000,211076.000000,211076.000000,2.317450e+05,231745.0,231745.0
mean,159060.633455,27.675167,28.076322,777.266590,0.458185,0.482070,0.006758,19.235582,-12.865614,1.442466e+08,100.0,0.0
std,64087.229135,33.455194,33.752894,5820.564791,0.283766,0.392335,0.009867,22.076527,21.631369,5.008530e+06,0.0,0.0
min,25000.000000,0.000000,0.010000,0.000000,0.151039,0.000619,0.000000,0.000223,-734.326500,1.305090e+08,100.0,0.0
25%,110000.000000,0.510000,0.590000,0.000000,0.310949,0.043089,0.001004,1.963998,-13.742892,1.414705e+08,100.0,0.0
50%,150000.000000,12.300000,12.650000,8.000000,0.371723,0.486595,0.003625,10.330390,-7.651854,1.447367e+08,100.0,0.0
75%,205000.000000,47.550000,48.250000,109.000000,0.484269,0.903945,0.008010,30.046845,-3.247278,1.481613e+08,100.0,0.0
max,320000.000000,156.700000,157.050000,365875.000000,2.998775,0.999998,0.184287,105.490200,-0.098726,1.517594e+08,100.0,0.0


Definition of the data:

1. [Strike Price](https://www.investopedia.com/terms/s/strikeprice.asp)
2. [Implied Volatility](https://www.investopedia.com/terms/i/iv.asp)
3. [Options Greeks](https://www.investopedia.com/trading/getting-to-know-the-greeks/#toc-delta)
4. [Forward Price](https://www.investopedia.com/terms/f/forwardprice.asp)

In [7]:
stock_prices_data.tail()

,Date,Open,High,Low,Close,Volume
246,01/07/2022,172.89,174.14,171.03,172.17,"86,709,148"
247,01/06/2022,172.70,175.30,171.64,172.00,"96,903,961"
248,01/05/2022,179.61,180.17,174.64,174.92,"94,537,602"
249,01/04/2022,182.63,182.94,179.12,179.70,"99,310,438"
250,01/03/2022,177.83,182.88,177.71,182.01,"104,701,203"


In [8]:
options_data['date'] = pd.to_datetime(options_data['date'])
stock_prices_data['Date'] = pd.to_datetime(stock_prices_data['Date'])

# Merge options and stock prices data based on date
merged_data = options_data.merge(stock_prices_data, left_on='date', right_on='Date', how="inner")

# Calculate time to expiration in years
merged_data['exdate'] = pd.to_datetime(merged_data['exdate'])
merged_data['time_to_expiration'] = (merged_data['exdate'] - merged_data['date']).dt.total_seconds() / (24 * 60 * 60 * 365)

columns_to_drop = ['Date', 'Open', 'High', 'Low', 'Volume', 'date', 'exdate', 'volume', 'delta', 'gamma', 'vega', 'theta', 'optionid', 'ticker', 'index_flag', 'issuer', 'exercise_style']
merged_data.drop(columns=columns_to_drop, inplace=True)

merged_data = merged_data.dropna()

# Filter call options
call_options_data = merged_data[merged_data['cp_flag'] == "C"]
call_options_data.drop(columns='cp_flag', inplace=True)

call_options_data = call_options_data.reset_index()

print("Processed Data Summary:")
call_options_data.head()

Processed Data Summary:


,index,last_date,strike_price,best_bid,best_offer,impl_volatility,contract_size,Close,time_to_expiration
0,18,2022-01-03,152500,28.25,31.25,0.937468,100,182.01,0.010959
1,20,2022-01-03,157500,24.20,24.95,0.630916,100,182.01,0.010959
2,22,2022-01-03,162500,19.30,20.65,0.753108,100,182.01,0.010959
3,26,2022-01-03,172500,9.40,9.65,0.196803,100,182.01,0.010959
4,27,2022-01-03,175000,7.10,7.20,0.247537,100,182.01,0.010959


In [9]:
train_call_options_data, test_call_options_data = train_test_split(call_options_data, test_size=0.1)
train_call_options_data, val_call_options_data = train_test_split(train_call_options_data, test_size=0.2)

In [10]:
train_call_options_data = train_call_options_data.reset_index(drop=True)
val_call_options_data = val_call_options_data.reset_index(drop=True)
test_call_options_data = test_call_options_data.reset_index(drop=True)

In [12]:
train_call_options_data.describe()

,index,strike_price,best_bid,best_offer,impl_volatility,contract_size,Close,time_to_expiration
count,145084.000000,145084.000000,145084.000000,145084.000000,145084.000000,145084.0,145084.000000,145084.000000
mean,116103.657660,164874.703620,24.045236,24.418945,0.450236,100.0,154.291475,0.558280
std,67103.541444,62091.977519,30.605068,30.912858,0.277543,0.0,12.993695,0.596709
min,18.000000,28750.000000,0.000000,0.010000,0.174370,100.0,126.040000,0.002740
25%,58199.750000,120000.000000,0.430000,0.500000,0.309642,100.0,143.780000,0.082192
50%,115289.500000,160000.000000,9.400000,9.700000,0.366649,100.0,153.040000,0.350685
75%,174583.250000,210000.000000,39.550000,40.150000,0.473899,100.0,165.350000,0.835616
max,231744.000000,320000.000000,152.850000,154.250000,2.998775,100.0,182.010000,2.350685


## Converting Data to Tensor and Data Loader

In [12]:
X_train = torch.tensor(train_call_options_data[['Close','time_to_expiration']].values, dtype=torch.float32)
y_train = torch.tensor(train_call_options_data.best_bid.values, dtype=torch.float32)

X_val = torch.tensor(val_call_options_data[['Close','time_to_expiration']].values, dtype=torch.float32)
y_val = torch.tensor(val_call_options_data.best_bid.values, dtype=torch.float32)

X_test = torch.tensor(test_call_options_data[['Close','time_to_expiration']].values, dtype=torch.float32)
y_test = torch.tensor(test_call_options_data.best_bid.values, dtype=torch.float32)

In [13]:
batch_size = 64

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


## Model Definition

In [14]:
## Defining the model
class MLPBlock(nn.Module):
    def __init__(self, hidden_size=64, num_layers=3, dropout_prob=0.5):
        super(MLPBlock, self).__init__()
        layers = []
        for _ in range(num_layers):
            layers.append(nn.Linear(hidden_size, hidden_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(p=dropout_prob))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

class NCLNetwork(nn.Module):
    def __init__(self, input_size=2, output_size=1, num_blocks=10, hidden_size=64, num_layers=3, dropout_prob=0.5):
        super(NCLNetwork, self).__init__()
        blocks = []
        for _ in range(num_blocks):
            blocks.append(MLPBlock(hidden_size, num_layers, dropout_prob))
            blocks.append(nn.BatchNorm1d(hidden_size))
            blocks.append(nn.Dropout(p=dropout_prob))
        self.input_layer = nn.Linear(input_size, hidden_size)
        self.blocks = nn.Sequential(*blocks)
        self.output_layer = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.input_layer(x)
        x = self.blocks(x)
        x = self.output_layer(x)
        return x

#Model Parameters
input_size = 2
output_size = 1
num_blocks = 10
hidden_size = 64
num_layers = 3
dropout_prob = 0.5

model = NCLNetwork(input_size, output_size, num_blocks, hidden_size, num_layers, dropout_prob)

for m in model.modules():
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)
        

## Model Training

In [15]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

# Define the learning rate range
base_lr = 0.00001  # The minimum learning rate
max_lr = 0.1     # The maximum learning rate

# Create a cyclic learning rate scheduler
clr_scheduler = CyclicLR(optimizer, base_lr=base_lr, max_lr=max_lr, step_size_up=2000, step_size_down=None, mode='triangular')


num_epochs = 50

In [18]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    average_train_loss = running_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:  
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

    average_val_loss = val_loss / len(val_loader)

    print(f'Epoch [{epoch + 1}/{num_epochs}] - Training Loss: {average_train_loss:.4f} - Validation Loss: {average_val_loss:.4f}')

Epoch [1/50] - Training Loss: 933.3686 - Validation Loss: 949.6721
Epoch [2/50] - Training Loss: 933.2621 - Validation Loss: 949.9557
Epoch [3/50] - Training Loss: 933.1835 - Validation Loss: 949.4250
Epoch [4/50] - Training Loss: 933.0832 - Validation Loss: 950.7198
Epoch [5/50] - Training Loss: 933.0891 - Validation Loss: 949.5595
Epoch [6/50] - Training Loss: 933.0696 - Validation Loss: 949.4720
Epoch [7/50] - Training Loss: 933.0878 - Validation Loss: 949.4714
Epoch [8/50] - Training Loss: 932.9890 - Validation Loss: 949.5244
Epoch [9/50] - Training Loss: 932.9836 - Validation Loss: 949.2025
Epoch [10/50] - Training Loss: 932.8775 - Validation Loss: 949.6137
Epoch [11/50] - Training Loss: 932.8452 - Validation Loss: 949.3332
Epoch [12/50] - Training Loss: 932.8657 - Validation Loss: 949.6141
Epoch [13/50] - Training Loss: 932.7643 - Validation Loss: 949.3760
Epoch [14/50] - Training Loss: 932.8026 - Validation Loss: 949.4601
Epoch [15/50] - Training Loss: 932.7111 - Validation Loss

In [19]:

# test dataset performance
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:  
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()

average_test_loss = test_loss / len(test_loader)
print(f'Test Loss: {average_test_loss:.4f}')


Test Loss: 946.5685
